In [1]:
import cv2
import numpy as np
import glob

# 체커보드 내부 코너 수 (정확히 맞춰야 함)
CHECKERBOARD = (9, 6)  # 예: 10x7 체스판이면 (9,6)

# 실제 사각형 한 칸의 크기 (단위: m, 예: 0.025m = 25mm)
square_size = 0.025

# 월드 좌표계에서 3D 좌표
objp = np.zeros((CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2) * square_size

objpoints = []  # 3D 점들
imgpoints = []  # 2D 점들

images = glob.glob('/home/cgh/workspace/project1/calibration/*.jpg')  # 또는 .jpg

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

        # 코너 시각화 (선택)
        cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

# ✅ gray 정의가 반복문에서 되고 있음
# 마지막 gray 사용 → 이미지 해상도용
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera Matrix:\n", mtx)
print("Distortion Coefficients:\n", dist)

# 저장 (선택)
# np.savez("/home/cgh/workspace/project1/calib_data.npz", mtx=mtx, dist=dist)


Camera Matrix:
 [[724.78566855   0.         322.13341418]
 [  0.         720.74252957 235.07954592]
 [  0.           0.           1.        ]]
Distortion Coefficients:
 [[ 1.55359498e-02  1.11160276e+00  6.03520232e-04  3.18295555e-03
  -3.77598061e+00]]


In [2]:
import numpy as np

data = np.load('/home/cgh/workspace/project1/calib_data.npz')
mtx = data['mtx']
dist = data['dist']

print("fx, fy =", mtx[0, 0], mtx[1, 1])
print("cx, cy =", mtx[0, 2], mtx[1, 2])
print("distortion =", dist)


fx, fy = 724.7856685470434 720.7425295731363
cx, cy = 322.13341417809346 235.07954592177197
distortion = [[ 1.55359498e-02  1.11160276e+00  6.03520232e-04  3.18295555e-03
  -3.77598061e+00]]
